# Project V. Fish Detection with Deep Learning
1. Split Train and Val dataset
2. Train a detection model based on YOLOv3-tiny
3. Evaluate your model
4. Use your model to detect fish from images in data/samples

## Setup
Please install required packages and make sure the version are valid 

pip install -r requirements.txt

In [1]:
from __future__ import division

from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.augmentations import *
from utils.transforms import *
from utils.parse_config import *
from utils.test import evaluate
from utils.loss import compute_loss
from utils.models import *

from terminaltables import AsciiTable
from matplotlib.ticker import NullLocator

import os
import sys
import time
import datetime
import argparse
import tqdm
from glob import glob

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim


# Data Preprocess
You should code this part first

In [2]:

# You should generate valid Train dataset and Val dataset.
# Use data in data/custom/images and data/custom/labels to generate the path file train.txt and 
# valid.txt in data/custom/
# a qualified val dataset is smaller than the train dataset and 
# most time there are no overlapped data between two sets.
# train.txt & valid.txt: one image per line
images_list = glob(os.path.join('.', 'data', 'custom', 'images', '*.jpg'))
train_txt_path = os.path.join('.', 'data', 'custom', 'train.txt')
val_txt_path = os.path.join('.', 'data', 'custom', 'valid.txt')

num_images = len(images_list)
train_set_size = int(num_images*0.6)
val_set_size = num_images - train_set_size

train_images_list = images_list[:train_set_size]
val_images_list = images_list[train_set_size:]

if os.path.isfile(train_txt_path):
    os.remove(train_txt_path)
if os.path.isfile(val_txt_path):
    os.remove(val_txt_path)

# im[2:] is for remove '.\' at the beginning, to match the format of given case
with open(train_txt_path,'w') as f:
    for im in train_images_list:
        f.write(f'{im[2:]}\n')

with open(val_txt_path,'w') as f:
    for im in val_images_list:
        f.write(f'{im[2:]}\n')



Make some config...

In [3]:
opt = {
    "epochs": 60,
    "model_def": "config/yolov3-tiny.cfg",
    "data_config": "config/custom.data",
    "pretrained_weights": "",
    "n_cpu": 1,
    "img_size": 416,
    "multiscale_training": True,
    "detect_image_folder": "data/samples"
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)  

# Get data configuration    
data_config = parse_data_config(opt["data_config"])    
train_path = data_config["train"]    
valid_path = data_config["valid"]    
class_names = load_classes(data_config["names"])
print(train_path)
print(valid_path)
print(class_names)

data/custom/train.txt
data/custom/valid.txt
['Fish']


use pytorch to generate our model and dataset

In [4]:
# Initiate model
model = Darknet(opt["model_def"]).to(device)
model.apply(weights_init_normal)

# If specified we start from checkpoint
if opt["pretrained_weights"] != "":
    if opt["pretrained_weights"].endswith(".pth"):
         model.load_state_dict(torch.load(opt["pretrained_weights"]))
    else:
         model.load_darknet_weights(opt["pretrained_weights"])

# Get dataloader
dataset = ListDataset(train_path, multiscale=opt["multiscale_training"], img_size=opt["img_size"], transform=AUGMENTATION_TRANSFORMS)
print(len(dataset.__getitem__(0)))
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size= model.hyperparams['batch'] // model.hyperparams['subdivisions'],
    shuffle=True,
    # num_workers=opt["n_cpu"],
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)

if (model.hyperparams['optimizer'] in [None, "adam"]):
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=model.hyperparams['learning_rate'],
        weight_decay=model.hyperparams['decay'],
        )
elif (model.hyperparams['optimizer'] == "sgd"):
    optimizer = torch.optim.SGD(
        model.parameters(), 
        lr=model.hyperparams['learning_rate'],
        weight_decay=model.hyperparams['decay'],
        momentum=model.hyperparams['momentum'])
else:
    print("Unknown optimizer. Please choose between (adam, sgd).")


3


# Train your model!
You are required to complete the DL project training steps (get data batch from dataloader, forward, compute the loss and backward)
see more details in following comments.

In [5]:
for epoch in range(opt["epochs"]):
#for epoch in range(2):
    print("\n---- Training Model ----")
    # activate BN and dropout
    model.train()

    # Your code need to execute forward and backward steps.
    # Use 'enumerate' to get a batch[_, images, targets]
    # some helpful function
    # - outputs = model.__call__(imgs)(use it by model(imgs))
    # - loss, _ = cumpte_loss(outputs, targets, model)
    # - loss.backward() (backward step)
    # - optimizer.step() (execute params updating)
    # - optimizer.zero_grad() (reset gradients)
    # if you want to see how loss changes in each mini-batch step:
    # -eg print(f'Epoch:{epoch+1}, Step{step+1}/{len(dataloader)}, loss:{loss.item()}')

    # each epoch
    
    for step, (_, im, targets) in enumerate(dataloader):
        if torch.cuda.is_available():
            im=im.type(torch.cuda.FloatTensor)
            targets=targets.type(torch.cuda.FloatTensor)
            
        y_pred = model.forward(im)
        loss, _ = compute_loss(y_pred, targets, model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch:{epoch+1}, Step{step+1}/{len(dataloader)}, loss:{loss.item()}')

        



---- Training Model ----
Epoch:1, Step1/72, loss:5.595037460327148
Epoch:1, Step2/72, loss:5.04945707321167
Epoch:1, Step3/72, loss:4.449802398681641
Epoch:1, Step4/72, loss:3.743774890899658
Epoch:1, Step5/72, loss:3.350261688232422
Epoch:1, Step6/72, loss:2.8249576091766357
Epoch:1, Step7/72, loss:2.871312141418457
Epoch:1, Step8/72, loss:2.1899731159210205
Epoch:1, Step9/72, loss:2.1420326232910156
Epoch:1, Step10/72, loss:1.7317065000534058
Epoch:1, Step11/72, loss:1.6434321403503418
Epoch:1, Step12/72, loss:1.4428608417510986
Epoch:1, Step13/72, loss:1.4120488166809082
Epoch:1, Step14/72, loss:1.8092010021209717
Epoch:1, Step15/72, loss:1.0434226989746094
Epoch:1, Step16/72, loss:0.9525754451751709
Epoch:1, Step17/72, loss:0.8846005797386169
Epoch:1, Step18/72, loss:0.829906165599823
Epoch:1, Step19/72, loss:0.6901243329048157
Epoch:1, Step20/72, loss:0.7900049090385437
Epoch:1, Step21/72, loss:0.7584354281425476
Epoch:1, Step22/72, loss:0.9188603162765503
Epoch:1, Step23/72, los

# Evaluate and save current model

In [6]:
print("\n---- Evaluating Model ----")
# Evaluate the model on the validation set
metrics_output = evaluate(
    model,
    path=valid_path,
    iou_thres=0.5,
    conf_thres=0.1,
    nms_thres=0.5,
    img_size=opt["img_size"],
    batch_size=model.hyperparams['batch'] // model.hyperparams['subdivisions'],
)

if metrics_output is not None:
    precision, recall, AP, f1, ap_class = metrics_output
    evaluation_metrics = [
                ("validation/precision", precision.mean()),
                ("validation/recall", recall.mean()),
                ("validation/mAP", AP.mean()),
                ("validation/f1", f1.mean()),
                ]
    # Print class APs and mAP
    ap_table = [["Index", "Class name", "AP"]]
    for i, c in enumerate(ap_class):
        print(class_names, c)
        ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
    print(AsciiTable(ap_table).table)
    print(f"---- mAP {AP.mean()}")                
else:
    print( "---- mAP not measured (no detections found by model)")
torch.save(model.state_dict(), f"checkpoints/yolov3-tiny_ckpt_%d.pth" % epoch)

Detecting objects:   0%|          | 0/49 [00:00<?, ?it/s]
---- Evaluating Model ----
Computing AP: 100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]['Fish'] 0
+-------+------------+---------+
| Index | Class name | AP      |
+-------+------------+---------+
| 0     | Fish       | 0.00000 |
+-------+------------+---------+
---- mAP 0.0



# Detect and visualize results

In [7]:
model.eval()  # Set in evaluation mode
dataloader = DataLoader(
        ImageFolder(opt["detect_image_folder"], transform= \
            transforms.Compose([DEFAULT_TRANSFORMS, Resize(opt["img_size"])])),
        batch_size=1,
        shuffle=False,
    )
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
imgs = []  # Stores image paths
img_detections = []  # Stores detections for each image index
print("\nPerforming object detection:")
for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))
    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = non_max_suppression(detections, 0.2, 0.7)
    imgs.extend(img_paths)
    img_detections.extend(detections)
# Bounding-box colors
cmap = plt.get_cmap("tab20b")
colors = [cmap(i) for i in np.linspace(0, 1, 20)]
print("\nSaving images:")
# Iterate through images and save plot of detections
for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):
    print("(%d) Image: '%s'" % (img_i, path))
    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    # Draw bounding boxes and labels of detections
    if detections is not None:
        # Rescale boxes to original image
        detections = detections.cpu()
        detections = rescale_boxes(detections, opt["img_size"], img.shape[:2])
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, cls_conf, cls_pred in detections:
            print("\t+ Label: %s, Conf: %.5f" % (class_names[int(cls_pred)], cls_conf.item()))
            box_w = x2 - x1
            box_h = y2 - y1
            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(
                x1,
                y1,
                s=class_names[int(cls_pred)],
                color="white",
                verticalalignment="top",
                bbox={"color": color, "pad": 0},
            )
    # Save generated image with detections
    plt.axis("off")
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    filename = os.path.basename(path).split(".")[0]
    output_path = os.path.join("output", f"{filename}.jpg")
    plt.savefig(output_path, bbox_inches="tight", pad_inches=0.0)
    plt.close()


Performing object detection:

Saving images:
(0) Image: 'data/samples\test (1).jpg'
(1) Image: 'data/samples\test (10).jpg'
(2) Image: 'data/samples\test (11).jpg'
(3) Image: 'data/samples\test (2).jpg'
	+ Label: Fish, Conf: 0.20867
	+ Label: Fish, Conf: 0.20627
(4) Image: 'data/samples\test (3).jpg'
(5) Image: 'data/samples\test (4).jpg'
(6) Image: 'data/samples\test (5).jpg'
(7) Image: 'data/samples\test (6).jpg'
(8) Image: 'data/samples\test (7).jpg'
(9) Image: 'data/samples\test (8).jpg'
(10) Image: 'data/samples\test (9).jpg'


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>